In [ ]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time 
import os
import matplotlib.pyplot as plt
import base64
import concurrent.futures

def scrape_data(url):
    data = {
        'merchant' : [],
        'category' : [],
        'star' : [],
        'address' : [],
        'oper_time' : [],
        'starCount' : [],
        'reviewCount' : []
    }
    data2 = {
        'user_name' : [],
        'user_rank' : [],
        'num_response' : [],
        'user_star' : [], 
        'time' : [],
        'rating' : [],
        'content' : []
    }

    driver = webdriver.Chrome()
    driver.get(url)
    driver.implicitly_wait(3)

    # 영업시간 더 보기 클릭
    try:
        oper_more = driver.find_elements(By.CSS_SELECTOR, 'a[data-logevent = "main_info,more_timeinfo"]')
        oper_more[0].click()
        time.sleep(2)  

    except:
        pass

    # 후기 더보기 클릭
    try:
        link_more = driver.find_element(By.CLASS_NAME, 'txt_more')
        while link_more.text == "후기 더보기":
            link_more = driver.find_element(By.CLASS_NAME, 'txt_more')
            if link_more.text != "후기 접기":
                link_more.click()
                time.sleep(0.3)
    except:
        pass
    time.sleep(0.1)
  
    soup = bs(driver.page_source, 'html.parser')

    test_image = driver.execute_script("return document.querySelectorAll('canvas')[1].toDataURL();")
    image_data = test_image.split(',')[1]
    image_data_decoded = base64.b64decode(image_data)

    test_image2 = driver.execute_script("return document.querySelectorAll('canvas')[2].toDataURL();")
    image_data2 = test_image2.split(',')[1]
    image_data_decoded2 = base64.b64decode(image_data2)

    driver.close()

    mer = soup.find('h2', {'class':'tit_location'}).get_text()
    data['merchant'].append(mer)
    data['category'].append(soup.find('span',{'class':'txt_location'}).get_text().split(':')[1].strip())
    data['address'].append(soup.find('span', {'class': 'txt_address'}).get_text().replace(" ", "").replace("\n", ""))

    if len(soup.find_all('span',{'class':'color_b'})) > 1:
        data['star'].append(soup.find_all('span', {'class':'color_b'})[0].get_text().strip('점'))
        data['starCount'].append(soup.find('a', {'class': 'link_evaluation'})['data-cnt'])
        data['reviewCount'].append(soup.find_all('span', {'class':'color_b'})[1].get_text().strip('개'))
    else:
        data['star'].append('0')
        data['starCount'].append('0')
        data['reviewCount'].append(soup.find('span', {'class':'color_b'}).get_text().strip('개'))

    if oper_more:    
        data['oper_time'].append(soup.find_all('ul', {'class':'list_operation'})[1].get_text().strip())
    else:
        data['oper_time'].append(soup.find('ul', {'class':'list_operation'}).get_text().strip().replace("\n", ""))

    reveiw_listed = soup.find_all('li', {'data-ismy':'false'})

    for j in range(len(reveiw_listed)):
        data2['user_name'].append(soup.find_all('span',{'class':'txt_username'})[j].get_text())
        data2['user_rank'].append(soup.find_all('span', {'class' : 'txt_badge'})[j].get_text().strip('레벨'))
        data2['num_response'].append(
            soup.find_all('div',{'class':'unit_info'})[j].find_all('span',{'class':'txt_desc'})[0].get_text()
        )
        data2['user_star'].append(
            soup.find_all('div',{'class':'unit_info'})[j].find_all('span',{'class':'txt_desc'})[1].get_text()
        )
        data2['time'].append(
            soup.find_all('div',{'class':'unit_info'})[j].find('span',{'class':'time_write'}).get_text()
        )
        data2['content'].append(
            soup.find_all('p', {'class':'txt_comment'})[j].get_text().strip('더보기')
        )

        style_attribute = soup.find_all('div', {'class':'star_info'})[j].find('span',{'class':'ico_star inner_star'})['style']
        width_percent = float(style_attribute.split(':')[1].strip('%;'))
        rating_out_of_five = width_percent / 20  
        data2['rating'].append(rating_out_of_five)

    df2 = pd.DataFrame(data2)
    df2.to_csv(f'./data2/{mer}.csv', index=False)

    with open(f'./image/{mer}1.png', 'wb') as f:
        f.write(image_data_decoded)

    with open(f'./image/{mer}2.png', 'wb') as f:
        f.write(image_data_decoded2)

    print(f"{mer}.csv 및 이미지 저장 완료")

    return data

def main():
    driver = webdriver.Chrome()
    driver.get('https://map.kakao.com/')
    time.sleep(2)

    searchbox = driver.find_element(By.ID, 'search.keyword.query')
    searchbox.send_keys('홍대 맛집'+ Keys.ENTER)
    time.sleep(3)

    list_url = []

    driver.find_element(By.ID, 'info.search.place.more').click()
    time.sleep(2)
    driver.find_element(By.ID, 'info.search.page.no1').click()
    time.sleep(2)

    for count in range(1):
        for inx in range(2, 6):
            moreview_element = driver.find_elements(By.CSS_SELECTOR, 'a[data-id="moreview"]')
            for i in range(len(moreview_element)):
                list_url.append(moreview_element[i].get_attribute('href'))
            driver.find_element(By.ID, f'info.search.page.no{inx}').click()
            if inx == 5:
                moreview_element = driver.find_elements(By.CSS_SELECTOR, 'a[data-id="moreview"]')
                for i in range(len(moreview_element)):
                    list_url.append(moreview_element[i].get_attribute('href'))
            time.sleep(2)
        driver.find_element(By.ID ,'info.search.page.next').click()
        time.sleep(2)

    driver.close()

    data_list = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = executor.map(scrape_data, list_url)
        for result in results:
            data_list.append(result)

    df = pd.DataFrame(data_list)
    df.to_csv("./data1/data.csv", index=False)

if __name__ == "__main__":
    main()


In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time 
import os
import matplotlib.pyplot as plt
import base64
import concurrent.futures

def scrape_data(url):
    data = {
        'merchant' : [],
        'category' : [],
        'star' : [],
        'address' : [],
        'oper_time' : [],
        'starCount' : [],
        'reviewCount' : []
    }
    data2 = {
        'user_name' : [],
        'user_rank' : [],
        'num_response' : [],
        'user_star' : [], 
        'time' : [],
        'rating' : [],
        'content' : []
    }

    driver = webdriver.Chrome()
    driver.get(url)
    driver.implicitly_wait(3)

    # 영업시간 더 보기 클릭
    try:
        oper_more = driver.find_elements(By.CSS_SELECTOR, 'a[data-logevent = "main_info,more_timeinfo"]')
        oper_more[0].click()
        time.sleep(2)  

    except:
        pass

    # 후기 더보기 클릭
    try:
        link_more = driver.find_element(By.CLASS_NAME, 'txt_more')
        while link_more.text == "후기 더보기":
            link_more = driver.find_element(By.CLASS_NAME, 'txt_more')
            if link_more.text != "후기 접기":
                link_more.click()
                time.sleep(0.3)
    except:
        pass
    time.sleep(0.1)
  
    soup = bs(driver.page_source, 'html.parser')

    test_image = driver.execute_script("return document.querySelectorAll('canvas')[1].toDataURL();")
    image_data = test_image.split(',')[1]
    image_data_decoded = base64.b64decode(image_data)

    test_image2 = driver.execute_script("return document.querySelectorAll('canvas')[2].toDataURL();")
    image_data2 = test_image2.split(',')[1]
    image_data_decoded2 = base64.b64decode(image_data2)

    driver.close()

    mer = soup.find('h2', {'class':'tit_location'}).get_text()
    data['merchant'].append(mer)
    data['category'].append(soup.find('span',{'class':'txt_location'}).get_text().split(':')[1].strip())
    data['address'].append(soup.find('span', {'class': 'txt_address'}).get_text().replace(" ", "").replace("\n", ""))

    if len(soup.find_all('span',{'class':'color_b'})) > 1:
        data['star'].append(soup.find_all('span', {'class':'color_b'})[0].get_text().strip('점'))
        data['starCount'].append(soup.find('a', {'class': 'link_evaluation'})['data-cnt'])
        data['reviewCount'].append(soup.find_all('span', {'class':'color_b'})[1].get_text().strip('개'))
    else:
        data['star'].append('0')
        data['starCount'].append('0')
        data['reviewCount'].append(soup.find('span', {'class':'color_b'}).get_text().strip('개'))

    if oper_more:    
        data['oper_time'].append(soup.find_all('ul', {'class':'list_operation'})[1].get_text().strip())
    else:
        data['oper_time'].append(soup.find('ul', {'class':'list_operation'}).get_text().strip().replace("\n", ""))

    reveiw_listed = soup.find_all('li', {'data-ismy':'false'})

    for j in range(len(reveiw_listed)):
        data2['user_name'].append(soup.find_all('span',{'class':'txt_username'})[j].get_text())
        data2['user_rank'].append(soup.find_all('span', {'class' : 'txt_badge'})[j].get_text().strip('레벨'))
        data2['num_response'].append(
            soup.find_all('div',{'class':'unit_info'})[j].find_all('span',{'class':'txt_desc'})[0].get_text()
        )
        data2['user_star'].append(
            soup.find_all('div',{'class':'unit_info'})[j].find_all('span',{'class':'txt_desc'})[1].get_text()
        )
        data2['time'].append(
            soup.find_all('div',{'class':'unit_info'})[j].find('span',{'class':'time_write'}).get_text()
        )
        data2['content'].append(
            soup.find_all('p', {'class':'txt_comment'})[j].get_text().strip('더보기')
        )

        style_attribute = soup.find_all('div', {'class':'star_info'})[j].find('span',{'class':'ico_star inner_star'})['style']
        width_percent = float(style_attribute.split(':')[1].strip('%;'))
        rating_out_of_five = width_percent / 20  
        data2['rating'].append(rating_out_of_five)

    df2 = pd.DataFrame(data2)
    df2.to_csv(f'./data2/{mer}.csv', index=False)

    with open(f'./image/{mer}1.png', 'wb') as f:
        f.write(image_data_decoded)

    with open(f'./image/{mer}2.png', 'wb') as f:
        f.write(image_data_decoded2)

    print(f"{mer}.csv 및 이미지 저장 완료")

    return data



In [17]:
def main():
    driver = webdriver.Chrome()
    driver.get('https://map.kakao.com/')
    time.sleep(2)

    searchbox = driver.find_element(By.ID, 'search.keyword.query')
    searchbox.send_keys('홍대 맛집'+ Keys.ENTER)
    time.sleep(3)

    list_url = []

    driver.find_element(By.ID, 'info.search.place.more').click()
    time.sleep(2)
    driver.find_element(By.ID, 'info.search.page.no1').click()
    time.sleep(2)

    for count in range(1):
        for inx in range(2, 3):
            moreview_element = driver.find_elements(By.CSS_SELECTOR, 'a[data-id="moreview"]')
            for i in range(9,12):
                list_url.append(moreview_element[i].get_attribute('href'))
            driver.find_element(By.ID, f'info.search.page.no{inx}').click()

            time.sleep(2)
        driver.find_element(By.ID ,'info.search.page.next').click()
        time.sleep(2)
    
    print(len(list_url))

    driver.close()

    data_list = []
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = executor.map(scrape_data, list_url)
        for result in results:
            data_list.append(result)

    df = pd.DataFrame(data_list)
    df.to_csv("./data1/data.csv", index=False)

if __name__ == "__main__":
    main()

3
또보겠지떡볶이집 깐따삐아점.csv 및 이미지 저장 완료
비스트로주라.csv 및 이미지 저장 완료


JavascriptException: Message: javascript error: Cannot read properties of undefined (reading 'toDataURL')
  (Session info: chrome=124.0.6367.61)
Stacktrace:
	GetHandleVerifier [0x00007FF7956C1522+60802]
	(No symbol) [0x00007FF79563AC22]
	(No symbol) [0x00007FF7954F7CE4]
	(No symbol) [0x00007FF7954FDE26]
	(No symbol) [0x00007FF7955004F6]
	(No symbol) [0x00007FF79558AEC3]
	(No symbol) [0x00007FF79556AB7A]
	(No symbol) [0x00007FF79558A224]
	(No symbol) [0x00007FF79556A923]
	(No symbol) [0x00007FF795538FEC]
	(No symbol) [0x00007FF795539C21]
	GetHandleVerifier [0x00007FF7959C413D+3217821]
	GetHandleVerifier [0x00007FF795A060D7+3488055]
	GetHandleVerifier [0x00007FF7959FF05F+3459263]
	GetHandleVerifier [0x00007FF79577B866+823494]
	(No symbol) [0x00007FF795645FBF]
	(No symbol) [0x00007FF795640EE4]
	(No symbol) [0x00007FF795641072]
	(No symbol) [0x00007FF7956318C4]
	BaseThreadInitThunk [0x00007FF9F2537344+20]
	RtlUserThreadStart [0x00007FF9F41E26B1+33]


In [3]:
a= pd.read_csv('./data1/data.csv')

In [12]:
a

,merchant,category,star,address,oper_time,starCount,reviewCount
0,['피오니 홍대점'],['카페'],['3.8'],['서울마포구독막로7길511층(우)04043'],['매일 12:00 ~ 21:00\n\n매일 라스트오더 ~ 20:30'],['274'],['642']
1,['943킹스크로스'],['테마카페'],['4.1'],"['서울마포구양화로16길24지하1층,1~4층(우)04039']",['월~목 11:30 ~ 21:30\n\n금~일 10:00 ~ 21:30'],['360'],['191']
2,['카미야'],"['돈까스,우동']",['4.2'],['서울마포구와우산로21길28-6지하1층(우)04040'],['매일 11:00 ~ 21:30\n\n매일 라스트오더 ~ 21:00'],['191'],['426']
3,['우와 홍대본점'],['일식'],['3.4'],['서울마포구와우산로21길21-162층202호(우)04041'],['매일 12:00 ~ 22:00월~금 휴게시간 14:30 ~ 17:00'],['168'],['624']
4,['칸다소바 홍대점'],['일본식라면'],['4.3'],['서울마포구와우산로51-61층(우)04049'],['매일 10:30 ~ 21:30\n\n월~토 휴게시간 16:00 ~ 17:00\n...,['279'],['439']


In [6]:
b = pd.read_csv('./data2/카미야.csv')

In [7]:
b

,user_name,user_rank,num_response,user_star,time,rating,content
0,김나영,8,24,5.0,2024.04.11.,5.0,NaN
1,내돈내산 솔직리뷰,26,63,3.5,2024.04.09.,3.0,돈까스 맛은 제 동네와 비슷한데 여긴 레몬즙이 있어서 뿌려먹으니 안물리고 맛있었어요...
2,냐옹,18,88,3.8,2024.04.08.,5.0,NaN
3,ㅇㅇ,3,1,2.0,2024.04.08.,2.0,웨이팅하고 오꼬노미야끼동 먹었어요. 맛있게 먹고 있었는데 나이 좀 있으신 여자분 오...
4,마포구 쩝쩝박사,33,89,3.3,2024.04.04.,4.0,엄청 뛰어난 맛은 아닌데 가격이 좋아요 요즘 기본 만오천 넘는데 치즈카츠 만원임
...,...,...,...,...,...,...,...
186,디카페인,31,134,4.1,2019.02.11.,4.0,냉모밀 짱짱 추천. 메뉴 전부 맛있지만 전체적으로 짠 편임.
187,.,10,9,3.3,2019.01.20.,5.0,치즈카츠랑 새우튀김을 정말 좋아하는데 아주 만족스러웠어요 그리고 크로켓을 추가로 주...
188,TBO,3,15,2.5,2018.12.10.,4.0,NaN
189,[CHAN],33,277,3.4,2018.09.25.,3.0,샐러드 5우동. 5히레. 2.5로스. 3.0서비스 굿기름이 별로인듯...
